## Usuarios con regiones de preferencia

Por cada item visto se amplia la región de preferencia del usuario con una nueva región centrada desde la metadata del contenido, con un radio inversamente proporcional al orden de tracks mas escuchados. 

$$R_\mu = \cup_{i=0}^N B|_{f(i)}(\mu\text{.top[i].vector}) \subset \Re^M$$

- N: Longitud del top del usuario.
- M: Numero de Audio Features.
- $f: \Re \rightarrow \Re$: Funcion que determina el radio de cada bola, con decaimiento (decreciente) a optimizar.

Sea $A(\mu, \lambda)$ la afinidad entre dos usuarios $\mu$ y $\lambda$, sabemos que está directamente relacionada con el valor del área de la intersección de sus regiones.

$$A(\mu, \lambda) \sim \text{Area}(R_\mu \cap R_\lambda)$$

#### Obtencion de usuarios similares...

- Intento 1:  Una opcion puede ser hacer $m<N$ queries al indice de tracks de Elastic, con el primer subconjunto de $m$ elementos de los tracks tops del usuario, pidiendo que los elementos devueltos esten a una distancia menor del radio correspondiente a cada track.  
 Luego, todos los tracks devueltos forman parte de la region de ese usuario $R_\mu$, entonces me fijo en el indice de usuarios qué usuarios tienen esos mismos elementos.
 Como desventaja, esto haria la busqueda una operacion muy costosa, ya que se deben hacer $m+2$ consultas a Elasticsearch.

In [1]:
import os
from dotenv import load_dotenv
import spotipy
from spotipy.oauth2 import SpotifyClientCredentials

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import plotly.express as px 

from sklearn.cluster import KMeans
from sklearn.manifold import TSNE
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline

import sys
os.chdir("..")
os.chdir("..")
sys.path.insert(0, os.getcwd())
from models.track import Track
from models.elasticLink import ElasticLink
from models.user import User

C:\Users\sebas\AppData\Local\Temp\ipykernel_24272\1101241361.py:7: DeprecationWarning: 
Pyarrow will become a required dependency of pandas in the next major release of pandas (pandas 3.0),
(to allow more performant data types, such as the Arrow string type, and better interoperability with other libraries)
but was not found to be installed on your system.
If this would cause problems for you,
please provide us feedback at https://github.com/pandas-dev/pandas/issues/54466
        
  import pandas as pd


In [2]:
# load .env file
load_dotenv()
elastic_endpoint = os.getenv("ELASTIC_HOST")
elastic_api_key = os.getenv("API_KEY")
spotify_client_id = os.getenv("SPOTIFY_CLIENT_ID")
spotify_client_secret = os.getenv("SPOTIFY_CLIENT_SECRET")

# elasticsearch link 
es = ElasticLink(elastic_endpoint, elastic_api_key)

# spotify
os.environ["SPOTIPY_CLIENT_ID"] = spotify_client_id
os.environ["SPOTIPY_CLIENT_SECRET"] = spotify_client_secret
os.environ['SPOTIPY_REDIRECT_URI']='http://localhost:8888/callback'
client_credentials_manager = SpotifyClientCredentials(client_id=spotify_client_id, client_secret=spotify_client_secret) 
sp = spotipy.Spotify(client_credentials_manager=client_credentials_manager)

### Carga de datos de prueba

In [6]:
from createTestUsers import getTestUsers
testUsers, allTracks, songsByArtist = getTestUsers()

# los agrego a la base de datos
for user in testUsers:
    user.getUserVectorFromListened()
    es.insertUser(user)

### Uso

In [56]:
def get_documents_within_radius(index, vector, radius):
    query = {
        "query": {
            "function_score": {
                "query": {
                    "bool": {
                        "must": {
                            "match_all": {},
                        }
                    }
                },
                "functions": [
                    {
                        "script_score": {
                            "script": {
                                "source": "l2norm(params.queryVector, 'metaVector') > params.radius ? 0 : 1 / (l1norm(params.queryVector, 'metaVector') + 1)",
                                "params": {
                                    "queryVector": vector,
                                    "radius": radius
                                }
                            }
                        }
                    }
                ]                                       
            }
        }
    }
    resp = es.client.search(index=index, body=query)
    return resp

In [57]:
r = get_documents_within_radius("users-metadata", allTracks[0].getMetadataVector(), 4)

In [58]:
def getSimilarUsersByRegion(user, m=2, f=lambda x: 10/(x+1)):
    # Check if m < N
    if len(user.listened) < m:
        raise Exception("User has not listened enough tracks")
    
    # Get all tracks in the user region
    regionTrackIds = set() 
    i = 0
    while i < m:
        track = user.listened[i]
        radiusTracks = get_documents_within_radius("songs-metadata", track.getMetadataVector(), f(i))
        for r in radiusTracks['hits']['hits']:
            if r['_score'] > 0:
                regionTrackIds.add(r['_source']['spotify_id'])
        i+=1 

    # Get all users that have listened to the tracks in the region
    query = {
        "size": 5,
        "query": {
            "function_score": {
                "query": {
                    "bool": {
                        "must": {
                            "match_all": {},
                        },
                        "must_not": [
                            {
                                "term": {
                                    "spotify_user_id": user.spotify_id
                                }
                            }
                        ],
                        "should": [
                            {
                                "terms": {
                                    "listened": list(regionTrackIds),
                                    "boost": 300
                                }
                            },
                            {
                                "terms": {
                                    "artists": list(user.artists),
                                    "boost": 200
                                }
                            }
                        ]
                    }
                },
            }
        }
    }
    resp = es.client.search(index="users-metadata", body=query)
    return resp

In [59]:
def testRegionRecm(u):
    print("Based on -->", u.spotify_id)
    resp = getSimilarUsersByRegion(u)['hits']['hits']
    for r in resp:
        print("-->", r['_source']['spotify_user_id'], r['_source']['artists'])
        
testRegionRecm(testUsers[1])

Based on --> Duki Fan
--> Taylor Swift and Emilia Fan ['TINI', 'Emilia', 'Taylor Swift', 'Nicki Nicole', 'WOS', 'Duki']
--> Taylor Swift Fan ['Taylor Swift']
--> The 1975 Fan ['The 1975']
--> House fan ['SIDEPIECE', 'Diplo', 'Sleepy Tom']
--> Emilia Fan ['TINI', 'Emilia']
